In [1]:
import os
from sympy import Poly,terms_gcd,gcd_list
import re
from sympy import factorint,fraction,primerange

primes=list(primerange(0, 1000))
print(len(primes))
#utility functions block

def encode(value):
    prefix=[]
    if abs(value) == 1:
        factors = {1: 1}
    else:
        factors = factorint(abs(value))
    remainder = value
    for i,fac in enumerate(factors.keys()):
        if int(fac) > 0:
            remainder = int(remainder / pow(fac,factors[fac]))
            if i != 0:
                if factors[fac] != 1: prefix += (f"*{fac}E{factors[fac]}")
                else: prefix += (f"*{fac}")
            else:
                if factors[fac] != 1: prefix += (f"{fac}E{factors[fac]}")
                else: prefix += (f"{fac}")   
    if remainder != 0:
        suffix = []
        w = abs(remainder)
        if w != 1:
            suffix.append(str(w))
    else:
        prefix = '0'
    prefix=''.join(prefix)
    prefix = ('+' if value >= 0 else '-') + prefix
    return ''.join(prefix)

def encodefrac(value):
    val=fraction(value)
    if val[1]=='1':
        prefix=encode(val[0])
    else:
        prefix=encode(val[0])+'/'+encode(val[1])
    prefix = ('+' if value >= 0 else '-') + prefix
    return prefix

def readESymb(name):
    assert os.path.isfile(name)
    res = ''
    prefix = 'all7_new_common_factor'

    with open(name, 'rt') as f:
        reading_form = False
        for line in f:
            if not reading_form:
                if not line.startswith(prefix): continue
                res = ''
                reading_form = True
            res += line[:-2] if line[-2] == '\\' else line[:-1]
            if line[-2] in [":", ";"]:
                break
    return res

def cl(text):
    return re.sub(r'\s+', '', text)
    
def convert(filename):
 
    #input_str = re.sub('\]=', '\] =', readESymb(filename))
    input_str = readESymb(filename)
    print(input_str)
    #input_str = re.sub('[,*]', '', readESymb(filename))
    # Use regular expressions to find all key-value pairs
    pattern = r'\[(.*?)\](.*?),'
    matches = re.findall(pattern, input_str)
    out_dict = {cl(re.sub(',', '', match[0])):cl(re.sub('=', '', match[1])) for match in matches}
    return out_dict




168


In [2]:
encode(143)

'+11*13'

In [3]:
from sympy.parsing.sympy_parser import (
    parse_expr,
    standard_transformations, 
    implicit_multiplication,
)

def parse(t):
    return parse_expr(re.sub('\^','**',t), transformations=standard_transformations + (implicit_multiplication,))


In [4]:
allpolys = convert('../data/all7_new_common_factor')
mypolys = {k:v for k, v in allpolys.items() if v != '0'}
uglypolys = {k:v for k, v in mypolys.items() if '(' not in v}

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
import math
mycoeffs={}
mygcds={}
mycoeffs_nogcd={}
mycoeffs_unenc={}
myquadrats={}
mylins={}
for k,v in mypolys.items():
    this_coeffs=Poly(parse(v).expand()).all_coeffs()
    mygcd=gcd_list(this_coeffs)
    mygcds[k]=mygcd
    if Poly(parse(v).expand()).degree('L') < 3:
        myquadrats[k] = this_coeffs
        this_coeffs = [0] + this_coeffs
        if Poly(parse(v).expand()).degree('L') < 2:
            mylins[k] = this_coeffs
            this_coeffs = [0] + this_coeffs
    myc=[c/mygcd for c in this_coeffs]
    mycoeffs_nogcd[k] = [c for c in this_coeffs]
    mycoeffs_unenc[k] = [[str(mygcd)],[c for c in myc]]
    mycoeffs[k] = [[str(mygcd)],[encode(c) for c in myc]]
    if len(this_coeffs) < 4:
        print(k,Poly(parse(v).expand()))

In [6]:
print(mycoeffs_nogcd)

{'aaaaabb': [-8/3, 136, -2584/3, 1440], 'aaaaabc': [8/3, -136, 2584/3, -1440], 'aaaaacb': [8/3, -136, 2584/3, -1440], 'aaaaacc': [-8/3, 136, -2584/3, 1440], 'aaaaaec': [768, -7600, 23712, -22400], 'aaaaafb': [768, -7600, 23712, -22400], 'aaaabab': [0, 120, -880, 1600], 'aaaabac': [0, -120, 880, -1600], 'aaaabbb': [4/3, -128, 2612/3, -1520], 'aaaabbc': [-4/3, 128, -2612/3, 1520], 'aaaabbd': [0, -128, 896, -1568], 'aaaabcb': [-4/3, 8, 28/3, -80], 'aaaabcc': [4/3, -8, -28/3, 80], 'aaaabcd': [0, 128, -896, 1568], 'aaaabdb': [0, -48, 320, -544], 'aaaabdc': [0, 48, -320, 544], 'aaaabfb': [0, -80, 608, -1152], 'aaaacab': [0, -120, 880, -1600], 'aaaacac': [0, 120, -880, 1600], 'aaaacbb': [4/3, -8, -28/3, 80], 'aaaacbc': [-4/3, 8, 28/3, -80], 'aaaacbd': [0, 128, -896, 1568], 'aaaaccb': [-4/3, 128, -2612/3, 1520], 'aaaaccc': [4/3, -128, 2612/3, -1520], 'aaaaccd': [0, -128, 896, -1568], 'aaaacdb': [0, 48, -320, 544], 'aaaacdc': [0, -48, 320, -544], 'aaaacec': [0, -80, 608, -1152], 'aaaaeab': [-57

In [7]:
from sympy import Rational
mydivs={k:v for k,v in mycoeffs.items() if ('/' in v[0][0])}
myints_unenc={k:mycoeffs_unenc[k] for k,v in mycoeffs.items() if ('/' not in v[0][0])}

my3div={k:mycoeffs[k] for k,v in mydivs.items() if (Rational(v[0][0]).q % 3 == 0) }
my3div_unenc={k:mycoeffs_unenc[k] for k,v in mydivs.items() if (Rational(v[0][0]).q % 3 == 0) }

my3inttop={k:mycoeffs[k] for k,v in myints_unenc.items() if (int(v[0][0]) % 3 == 0)}
my3inttop_unenc={k:mycoeffs_unenc[k] for k,v in myints_unenc.items() if (int(v[0][0]) % 3 == 0)}

my2inttop={k:mycoeffs[k] for k,v in myints_unenc.items() if (int(v[0][0]) % 2 == 0)}
my2inttop_unenc={k:mycoeffs_unenc[k] for k,v in myints_unenc.items() if (int(v[0][0]) % 2 == 0)}

my5div={k:mycoeffs[k] for k,v in mydivs.items() if (Rational(v[0][0]).q % 5 == 0) }
my5div_unenc={k:mycoeffs_unenc[k] for k,v in mydivs.items() if (Rational(v[0][0]).p % 5 == 0) }

my5inttop={k:mycoeffs[k] for k,v in myints_unenc.items() if (int(v[0][0]) % 5 == 0)}
my5inttop_unenc={k:mycoeffs_unenc[k] for k,v in myints_unenc.items() if (int(v[0][0]) % 5 == 0)}

In [91]:
from sympy import Rational
from sympy import Rational

def enc_elem(elem):
    if '/' in str(elem):
        return str(encode(elem.p))+'/'+str(encode(elem.q))
    else: return str(encode(elem))

def is_pow2(n):
    return (n != 0) and (n & (n-1) == 0)

def showcoeffs(i):
    mydivs={k:v for k,v in mycoeffs_nogcd.items() if len(v) > i and ('/' in str(v[i]))}
    myints={k:mycoeffs_nogcd[k] for k,v in mycoeffs_nogcd.items() if len(v) > i and ('/' not in str(v[i]))}
    
    mydivs_enc={k:[enc_elem(coef) for coef in mycoeffs_nogcd[k]] for k,v in mydivs.items()}
    myints_enc={k:[enc_elem(coef) for coef in mycoeffs_nogcd[k]] for k,v in myints.items()}

    mydivs_enc_sel={k:enc_elem(mycoeffs_nogcd[k][i]) for k,v in mydivs.items()}
    myints_enc_sel={k:enc_elem(mycoeffs_nogcd[k][i]) for k,v in myints.items()}

    #myints_enc={k:[str(encode(elem)) for elem in mycoeffs_nogcd[k]] for k,v in mycoeffs_nogcd.items() if ('/' not in str(v[i]))}
    
    my3div={k:mydivs[k] for k,v in mydivs.items() if (Rational(v[i]).q % 3 == 0) }
    my3div_enc={k:mydivs_enc[k] for k in my3div}

    my5div={k:mydivs[k] for k,v in mydivs.items() if (Rational(v[i]).q % 5 == 0) }
    my5div_enc={k:mydivs_enc[k] for k in my5div}
    
    my2int={k:myints[k] for k,v in myints.items() if (int(v[i]) % 2 == 0)}
    my2int_enc={k:myints_enc[k] for k,v in myints.items() if (int(v[i]) % 2 == 0)}
    
    mypow2int={k:myints[k] for k,v in myints.items() if is_pow2(abs(int(v[i])))}
    mypow2int_enc={k:myints_enc[k] for k,v in myints.items() if is_pow2(abs(int(v[i])))}
    
    my3int={k:myints[k] for k,v in myints.items() if (int(v[i]) % 3 == 0)}
    my3int_enc={k:myints_enc[k] for k,v in myints.items() if (int(v[i]) % 3 == 0)}
    
    my5int={k:myints[k] for k,v in myints.items() if (int(v[i]) % 5 == 0)}
    my5int_enc={k:myints_enc[k] for k,v in myints.items() if (int(v[i]) % 5 == 0)}
    
    my_all=mydivs|myints
    
    outdict= {'all':my_all, 'div':mydivs,'int':myints, 'divs_enc':mydivs_enc, 'ints_enc':myints_enc,
              'divs_enc_sel':mydivs_enc_sel, 'ints_enc_sel':myints_enc_sel,
              '2int':my2int,'2int_enc':my2int_enc,'pow2int':mypow2int,'pow2int_enc':mypow2int_enc,
              '3div':my3div,'3div_enc':my3div_enc,'5div':my5div,'5div_enc':my5div_enc,
              '3int':my3int,'3int_enc':my3int_enc,'5int':my5int,'5int_enc':my5int_enc}
            
    return outdict

In [156]:
import sympy
myshow=showcoeffs(0)

#All keys where A,C are div by 45 end in abc.
#If they contain d,e,f it is a run of length <= 3 starting in the second slot.



d={k:v for k,v in myshow['div'].items() if (v[0].q == 3) and ('d' in k)}
d

{'aaababd': [-32/3, -16, 1520/3, -1120],
 'aaabacd': [32/3, 16, -1520/3, 1120],
 'aaabbbd': [16/3, 72, -2104/3, 1344],
 'aaabbcd': [-16/3, -72, 2104/3, -1344],
 'aaabbdb': [-8/3, 24, -232/3, 96],
 'aaabbdc': [-8/3, -24, 728/3, -448],
 'aaabbdd': [32/3, 16, -1328/3, 896],
 'aaabcbd': [16/3, -56, 584/3, -224],
 'aaabccd': [-16/3, 56, -584/3, 224],
 'aaabcdb': [8/3, -24, 232/3, -96],
 'aaabcdc': [8/3, 24, -728/3, 448],
 'aaabcdd': [-32/3, -16, 1328/3, -896],
 'aaacabd': [32/3, 16, -1520/3, 1120],
 'aaacacd': [-32/3, -16, 1520/3, -1120],
 'aaacbbd': [-16/3, 56, -584/3, 224],
 'aaacbcd': [16/3, -56, 584/3, -224],
 'aaacbdb': [8/3, 24, -728/3, 448],
 'aaacbdc': [8/3, -24, 232/3, -96],
 'aaacbdd': [-32/3, -16, 1328/3, -896],
 'aaaccbd': [-16/3, -72, 2104/3, -1344],
 'aaacccd': [16/3, 72, -2104/3, 1344],
 'aaaccdb': [-8/3, -24, 728/3, -448],
 'aaaccdc': [-8/3, 24, -232/3, 96],
 'aaaccdd': [32/3, 16, -1328/3, 896],
 'aaaeabd': [368/3, -1288, 13432/3, -5152],
 'aaaeacd': [-368/3, 1288, -13432/3,

In [ ]:
from sympy import Rational
from sympy import Rational
def showcoeffs(i):
    mydivs={k:v for k,v in mycoeffs_nogcd.items() if ('/' in str(v[i]))}
    myints={k:mycoeffs_nogcd[k] for k,v in mycoeffs_nogcd.items() if ('/' not in str(v[i]))}

    mydivs_enc={k:str(encode((mycoeffs_nogcd[k]).p))+'/'+str(encode((mycoeffs_nogcd[k]).q)) for k,v in mycoeffs_nogcd.items() if ('/' in str(v[i]))}
    myints_enc={k:str(encode((mycoeffs_nogcd[k]))) for k,v in mycoeffs_nogcd.items() if ('/' not in str(v[i]))}
    
    my3div={k:mydivs[k] for k,v in mydivs.items() if (Rational(v[i]).q % 3 == 0) }
    my3div_enc={k:mydivs_enc[k] for k in my3div}

    my5div={k:mydivs[k] for k,v in mydivs.items() if (Rational(v[i]).q % 5 == 0) }
    my5div_enc={k:mydivs_enc[k] for k in my5div}
    
    my2int={k:myints[k] for k,v in myints.items() if (int(v[i]) % 2 == 0)}
    my2int_enc={k:myints_enc[k] for k,v in myints.items() if (int(v[i]) % 2 == 0)}
    
    
    my3int={k:myints[k] for k,v in myints.items() if (int(v[i]) % 3 == 0)}
    my3int_enc={k:myints_enc[k] for k,v in myints.items() if (int(v[i]) % 3 == 0)}
    
    my5int={k:myints[k] for k,v in myints.items() if (int(v[i]) % 5 == 0)}
    my5int_enc={k:myints_enc[k] for k,v in myints.items() if (int(v[i]) % 5 == 0)}

    
    return mydivs,myints_unenc,my3div,my3div_unenc,my5div,my5div_unenc,my2inttop,my2inttop_unenc,my3inttop,my3inttop_unenc,my5inttop,my5inttop_unenc

In [ ]:
{k:v for k,v in mycoeffs.items() if k not in (my3inttop_unenc | my5inttop_unenc | mydivs | my2inttop_unenc)}



In [31]:
{v[0][0] for v in my3inttop.values()}

{'12', '24', '48', '6', '96'}

In [8]:
#all polynoms have a GCD that is a multiple of  (1/5), (1/3), (3), (5), a power of 2, or two or more of these.
#if GCD is 1, leading term is always divisible by 5.
#We see (1/3)^2 and (1/3)*(1/5) but NOT (1/5)^2
#The space of all GCDs is:
"""
 =(1/3)*{{1,2,4,8,16,32,64} + {44,184}}
 +(5/3)*{4,8,16}
 +(1/5)*{1,2,4,8}
 +(1/9)*{1,2,4,8,16,32}
 +(5/9)*{4}
 +(1/15)*{1,2,4,8,16,32}
 +(1/45)*{{1,2,4,8,16} + {28}}
 +{1,2,4,8,16,32,64,128}
 +(3)*{2,4,8,16,32}
 +(5)*{4,8}
"""
#This looks awfully combinatorial...

NameError: name 'my3divtop' is not defined

In [ ]:
#multiply it all by 

In [ ]:
#print([v[1][3] for k,v in my3div_unenc.items()])


{k:my5div_unenc[k][1] for k,v in my5div_unenc.items() if any([elem == 0 for elem in v[1]])}

In [ ]:
#all polynoms with no multiplicity in coefs have GCD as a multiple of 4/3

In [ ]:
# if the gcd is a multiple of (1/5), the constant term is either divisible by 20=5*4 or by 35=5*7
# if the gcd is a multiple of (1/3), not true!
#If GCD is multiple of 1/3, can have zero in L2 of L3 terms, not if L=5

In [ ]:
# if gcd is a multiple of 5, the key does not contain e,f.

In [ ]:
#GCD if rational is multiple of 1/3 or 1/5. Can be divisible by 1/9 but not 1/25

{k:v for k,v in mydivs.items() if (k in my5int)}


In [ ]:
factorpolys={k:v for k,v in mypolys.items() if k not in uglypolys}
L4factors={k:v for k,v in factorpolys.items() if '(L-4)' in v}
otherfactors={k:v for k,v in factorpolys.items() if k not in L4factors}

In [ ]:
print(len(L4factors))
print(len(otherfactors))
print(len(uglypolys))
print(len(mypolys))
badquads={k:v for k,v in myquadrats.items() if k in uglypolys}
badquads

In [ ]:
#There are 200 quadratic polynomials that do not factor nicely and 1430 that do.
#There are 1590 polynomials that do not factor nicely. 200 are quadratic, and 1390 are cubic.

In [ ]:
mycoeffs={}
myquadrats={}

for k,v in badquads.items():
    #print(k,v)
    this_coeffs=Poly(parse(v).expand()).all_coeffs()
    if Poly(parse(v).expand()).degree('L') < 3:
        badquads[k] = this_coeffs
        this_coeffs = [0] + this_coeffs
    badquadcoeffs[k] = this_coeffs